This cell provides an interactive interface for inspecting and editing the textual fields of the product data. Use it to review, modify, or complement product information as needed. All changes can be saved directly, making it easy to keep the product dataset up to date.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from html import escape

# Load the data
product_df = pd.read_json("../../Data/products.json", orient='index')
output_path = "../../Data/products.json"  # Same as input path

# Initialize index counter
current_index = 0

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Function to save the dataframe back to JSON
def save_data():
    product_df.to_json(output_path, orient='index', indent=2)

# Function to convert special characters for display
def display_format(text):
    if not isinstance(text, str):
        return str(text)
    # Convert special chars to visible format
    return (text)

# Function to convert back to original format for saving
def save_format(text):
    if not isinstance(text, str):
        return text
    # Convert back to original special chars
    return (text)

# Function to display current product
def display_product(index):
    global current_index, product_df
    
    # Ensure index is within bounds
    if index >= len(product_df):
        index = len(product_df) - 1
    if index < 0:
        index = 0
    current_index = index
    
    # Clear previous output
    clear_output(wait=True)
    
    # Display index name
    index_name = product_df.index[current_index]
    display(widgets.HTML(f"<h3>Product Index: {escape(index_name)}</h3>"))
    display(widgets.HTML(f"<p>Product {current_index + 1} of {len(product_df)}</p>"))
    
    # Get the current product data
    product_data = product_df.iloc[current_index].to_dict()
    
    # Create widgets for each attribute
    widgets_dict = {}
    for attr, value in product_data.items():
        # Convert value to display format
        display_value = display_format(value)
        
        # Create a text area with proper sizing for all fields
        widgets_dict[attr] = widgets.Textarea(
            value=display_value,
            description=attr,
            layout=widgets.Layout(width='90%', height='150px'),
            style={'description_width': 'initial'}
        )
    
    # Display all attribute widgets in an accordion for better organization
    accordion = widgets.Accordion(children=[widgets.VBox(list(widgets_dict.values()))])
    accordion.set_title(0, 'Product Attributes')
    accordion.selected_index = 0  # Start expanded
    display(accordion)
    
    # Create navigation buttons with larger size
    next_button = widgets.Button(description="Next", layout=widgets.Layout(width='150px'))
    prev_button = widgets.Button(description="Previous", layout=widgets.Layout(width='150px'))
    save_button = widgets.Button(
        description="Save Now", 
        button_style='success',
        layout=widgets.Layout(width='150px')
    )
    
    # Button click handlers
    def on_next_click(b):
        # Save current changes (converting back to original format)
        for attr, widget in widgets_dict.items():
            original_value = save_format(widget.value)
            product_df.at[product_df.index[current_index], attr] = original_value
        save_data()
        # Move to next product
        display_product(current_index + 1)
    
    def on_prev_click(b):
        # Save current changes (converting back to original format)
        for attr, widget in widgets_dict.items():
            original_value = save_format(widget.value)
            product_df.at[product_df.index[current_index], attr] = original_value
        save_data()
        # Move to previous product
        display_product(current_index - 1)
    
    def on_save_click(b):
        # Save current changes (converting back to original format)
        for attr, widget in widgets_dict.items():
            original_value = save_format(widget.value)
            product_df.at[product_df.index[current_index], attr] = original_value
        save_data()
        display(widgets.HTML("<p style='color:green;'>Changes saved!</p>"))
    
    # Assign handlers
    next_button.on_click(on_next_click)
    prev_button.on_click(on_prev_click)
    save_button.on_click(on_save_click)
    
    # Display buttons in a horizontal layout with some spacing
    buttons = widgets.HBox(
        [prev_button, next_button, save_button],
        layout=widgets.Layout(justify_content='center', margin='20px 0')
    )
    display(buttons)

# Start displaying the first product
idx=0
display_product(idx)

HTML(value='<h3>Product Index: Flexline and Qsonic</h3>')

HTML(value='<p>Product 386 of 1829</p>')

Accordion(children=(VBox(children=(Textarea(value='None', description='Bulletpoints', layout=Layout(height='15…